In [3]:
# Dependencies
!pip install mtcnn
!pip install git+https://github.com/hukkelas/DSFD-Pytorch-Inference.git
!pip install mediapipe

import cv2
import time
import face_detection
import mediapipe as mp
from mtcnn.mtcnn import MTCNN
from matplotlib import pyplot as plt
import numpy as np

%matplotlib inline


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/hukkelas/DSFD-Pytorch-Inference.git to /tmp/pip-req-build-dz3konkw
  Running command git clone -q https://github.com/hukkelas/DSFD-Pytorch-Inference.git /tmp/pip-req-build-dz3konkw
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
!git clone https://github.com/lorenzocollodi/face_detection_eval.git
!mv face_detection_eval/Annotation . 
!mv face_detection_eval/Data . 
!rm -rf face_detection_eval
!cp /content/Data/Ethnics/Caucasian/13_Interview_Interview_On_Location_13_166.jpg /content/test.jpg

Cloning into 'face_detection_eval'...
remote: Enumerating objects: 634, done.
remote: Counting objects: 100% (19/19), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 634 (delta 12), reused 6 (delta 6), pack-reused 615
Receiving objects: 100% (634/634), 82.67 MiB | 19.43 MiB/s, done.
Resolving deltas: 100% (56/56), done.


In [2]:
# Make models
# SSD
!wget "https://github.com/spmallick/learnopencv/blob/master/FaceDetectionComparison/models/res10_300x300_ssd_iter_140000_fp16.caffemodel?raw=true" -O res10_300x300_ssd_iter_140000_fp16.caffemodel
!wget "https://raw.githubusercontent.com/spmallick/learnopencv/master/FaceDetectionComparison/models/deploy.prototxt" -O deploy.prototxt
detector_SSD = cv2.dnn.DetectionModel("res10_300x300_ssd_iter_140000_fp16.caffemodel", "deploy.prototxt")

# MTCNN
detector_MTCNN = MTCNN()

# DSFD
detector_DSFD = face_detection.build_detector("DSFDDetector", confidence_threshold=.5, nms_iou_threshold=.3)

# Retina
detector_Retina = face_detection.build_detector("RetinaNetResNet50", confidence_threshold=.5, nms_iou_threshold=.3)

# MediaPipa
mp_face_detection = mp.solutions.face_detection
detector_MP = mp_face_detection.FaceDetection(model_selection=1, min_detection_confidence=0.4)

# YuNet
!wget "https://github.com/opencv/opencv_zoo/blob/master/models/face_detection_yunet/face_detection_yunet_2022mar.onnx?raw=true" -O face_detection_yunet_2022mar.onnx
detector_YN = cv2.FaceDetectorYN.create("face_detection_yunet_2022mar.onnx", "", (320, 320))

detectors = {"SSD": detector_SSD, "MTCNN": detector_MTCNN, "DSFD": detector_DSFD, "Retina": detector_Retina, "MP": detector_MP, "YuNet": detector_YN}

--2022-11-28 22:04:38--  https://github.com/spmallick/learnopencv/blob/master/FaceDetectionComparison/models/res10_300x300_ssd_iter_140000_fp16.caffemodel?raw=true
Resolving github.com (github.com)... 140.82.121.3
Connecting to github.com (github.com)|140.82.121.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/spmallick/learnopencv/raw/master/FaceDetectionComparison/models/res10_300x300_ssd_iter_140000_fp16.caffemodel [following]
--2022-11-28 22:04:38--  https://github.com/spmallick/learnopencv/raw/master/FaceDetectionComparison/models/res10_300x300_ssd_iter_140000_fp16.caffemodel
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/spmallick/learnopencv/master/FaceDetectionComparison/models/res10_300x300_ssd_iter_140000_fp16.caffemodel [following]
--2022-11-28 22:04:38--  https://raw.githubusercontent.com/spmallick/learnopencv/master/FaceDetectionC

Downloading: "https://folk.ntnu.no/haakohu/WIDERFace_DSFD_RES152.pth" to /root/.cache/torch/hub/checkpoints/WIDERFace_DSFD_RES152.pth


  0%|          | 0.00/459M [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
Downloading: "https://folk.ntnu.no/haakohu/RetinaFace_ResNet50.pth" to /root/.cache/torch/hub/checkpoints/RetinaFace_ResNet50.pth


  0%|          | 0.00/104M [00:00<?, ?B/s]

--2022-11-28 22:04:59--  https://github.com/opencv/opencv_zoo/blob/master/models/face_detection_yunet/face_detection_yunet_2022mar.onnx?raw=true
Resolving github.com (github.com)... 140.82.121.4
Connecting to github.com (github.com)|140.82.121.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/opencv/opencv_zoo/raw/master/models/face_detection_yunet/face_detection_yunet_2022mar.onnx [following]
--2022-11-28 22:04:59--  https://github.com/opencv/opencv_zoo/raw/master/models/face_detection_yunet/face_detection_yunet_2022mar.onnx
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://media.githubusercontent.com/media/opencv/opencv_zoo/master/models/face_detection_yunet/face_detection_yunet_2022mar.onnx [following]
--2022-11-28 22:04:59--  https://media.githubusercontent.com/media/opencv/opencv_zoo/master/models/face_detection_yunet/face_detection_yunet_2022mar.onnx
Resolving media.

In [ ]:
# Measure time on all the methods. To test on GPU set the runtime

for name, detector in detectors.items():
    for size in ((1, 2, 4, 8, 16)):
      # Take a random test image (testing on noise might be unfair to some methods)
      img = cv2.imread("/content/test.jpg")
      img = cv2.resize(img, (320*size, 240*size))
      if hasattr(detector, "detect"):
        img_W = int(img.shape[1])
        img_H = int(img.shape[0])
        if hasattr(detector, "setInputSize"):
          detector.setInputSize((img_W, img_H))
        detector.detect(img)
      elif hasattr(detector, "process"):
        detector.process(img)
      elif hasattr(detector, "detect_faces"):
        detector.detect_faces(img)
      else:
        detector(img)
      start_time = time.time()
      for i in range(10):
        if hasattr(detector, "detect"):
          img_W = int(img.shape[1])
          img_H = int(img.shape[0])
          detector.detect(img)
        elif hasattr(detector, "process"):
          detector.process(img)
        elif hasattr(detector, "detect_faces"):
          detector.detect_faces(img)
        else:
          detector(img)
      elapsed = (time.time() - start_time)/10
      print(f"{name}: evaluated size {size} in {elapsed}s, with {1/elapsed} FPS")

YuNet: evaluated size 1 in 0.011927962303161621s, with 83.83661639632616 FPS
YuNet: evaluated size 2 in 0.045400595664978026s, with 22.026142726832084 FPS
YuNet: evaluated size 4 in 0.17181038856506348s, with 5.820369817866435 FPS


In [ ]:
print(detectors["YuNet"].detect_faces(img))

AttributeError: ignored

In [8]:
import os

# Get all the images listed in a file, images' names are every three elements in the lists
annotation_path = "/content/Annotation"
data_path = "/content/Data"

files = ["annotated_accessories.txt", "annotated_ethnics.txt", "annotated_light.txt", "annotated_pose.txt", "annotated_expression.txt"]
imgs = set()
for f_name in files:
  for img in list(open(os.path.join(annotation_path, f_name), 'r').readlines())[::3]:
    imgs.add(img)
print(imgs)

{'Expression/Normal/30_Surgeons_Surgeons_30_861.jpg\n', 'Expression/Extreme/18_Concerts_Concerts_18_828.jpg\n', 'Accessories/No/18_Concerts_Concerts_18_655.jpg\n', 'Pose/Extreme/focus_00223.jpg\n', 'Light/Irregular/illumination_02593.jpg\n', 'Light/Irregular/illumination_00718.jpg\n', 'Light/Regular/30_Surgeons_Surgeons_30_861.jpg\n', 'Pose/Extreme/47_Matador_Bullfighter_Matador_Bullfighter_47_610.jpg\n', 'Ethnics/Asian/illumination_00695.jpg\n', 'Expression/Extreme/motion_02677.jpg\n', 'Ethnics/African/9_Press_Conference_Press_Conference_9_767.jpg\n', 'Ethnics/Caucasian/illumination_00057.jpg\n', 'Expression/Normal/36_Football_americanfootball_ball_36_301.jpg\n', 'Pose/Extreme/motion_00868.jpg\n', 'Expression/Normal/illumination_00336.jpg\n', 'Accessories/Yes/13_Interview_Interview_Sequences_13_779.jpg\n', 'Pose/Extreme/rain_02310.jpg\n', 'Accessories/No/16_Award_Ceremony_Awards_Ceremony_16_474.jpg\n', 'Expression/Extreme/41_Swimming_Swimming_41_283.jpg\n', 'Accessories/No/33_Running_

In [ ]:
models_to_test = ["MP", "Retina", "DSFD", "YuNet"]

for model in models_to_test:
  new_file_name = model + "_predictions.txt"
  new_file_lines = []
  new_file_path = os.path.join(data_path, new_file_name)
  detector = detectors[model]
  for line in imgs:
    img_path = os.path.join(data_path, line)
    print(img_path)
    img = cv2.imread(img_path.rstrip())
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    w = int(img.shape[1])
    h = int(img.shape[0])
    if max(h, w) > 1080:
      scale = 1080/max(h, w)
      img = cv2.resize(img, (int(w*scale), int(h*scale)))
    else:
      scale = 1
    if hasattr(detector, "detect"):
      img_W = int(img.shape[1])
      img_H = int(img.shape[0])
      if hasattr(detector, "setInputSize"):
        detector.setInputSize((img_W, img_H))
        _, prediction = detector.detect(img)
      else:
        prediction = detector.detect(img)
        for face in prediction:
          face[2] -= face[0]
          face[3] -= face[1]
    elif hasattr(detector, "process"):
      tmp_prediction = detector.process(img).detections
      if tmp_prediction is not None:
        prediction = []
        for face in tmp_prediction:
          BB = face.location_data.relative_bounding_box
          prediction += [[BB.xmin*w*scale, BB.ymin*h*scale, (BB.width)*w*scale, (BB.height)*h*scale]]
      else:
        prediction = tmp_prediction
    elif hasattr(detector, "detect_faces"):
      prediction = detector.detect_faces(img)
      for face in prediction:
        face[2] -= face[0]
        face[3] -= face[1]
    else:
      prediction = detector(img)
    if prediction is None:
      prediction =[]
    new_file_lines += line.rstrip() + '\n'
    new_file_lines += str(len(prediction)) + '\n'
    for face in prediction:
      new_file_lines += ' '.join([str(int(float(element)/scale)) for element in face[:4]]) + '\n'
  open(new_file_path, 'w').writelines(new_file_lines)

/content/Data/Expression/Normal/30_Surgeons_Surgeons_30_861.jpg

/content/Data/Expression/Extreme/18_Concerts_Concerts_18_828.jpg

/content/Data/Accessories/No/18_Concerts_Concerts_18_655.jpg

/content/Data/Pose/Extreme/focus_00223.jpg

/content/Data/Light/Irregular/illumination_02593.jpg

/content/Data/Light/Irregular/illumination_00718.jpg

/content/Data/Light/Regular/30_Surgeons_Surgeons_30_861.jpg

/content/Data/Pose/Extreme/47_Matador_Bullfighter_Matador_Bullfighter_47_610.jpg

/content/Data/Ethnics/Asian/illumination_00695.jpg

/content/Data/Expression/Extreme/motion_02677.jpg

/content/Data/Ethnics/African/9_Press_Conference_Press_Conference_9_767.jpg

/content/Data/Ethnics/Caucasian/illumination_00057.jpg

/content/Data/Expression/Normal/36_Football_americanfootball_ball_36_301.jpg

/content/Data/Pose/Extreme/motion_00868.jpg

/content/Data/Expression/Normal/illumination_00336.jpg

/content/Data/Accessories/Yes/13_Interview_Interview_Sequences_13_779.jpg

/content/Data/Pose/Ext